In [ ]:
import os

from collections import defaultdict

from DataLoader import (
    config,
    loader
)

import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns
 
from sklearn.preprocessing import (
    MinMaxScaler,
    PolynomialFeatures
)
from sklearn.decomposition import PCA

In [ ]:
def compress(signals: pd.DataFrame, floor: str='30min', method='max'):
    """
    floor: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
    method: Определяет каким образом сжимается ряд. Принимает значения 'max' - максимум интервала, 'mean' - среднее значение интервала, 'mixed' - E(x) + max()
    """
    match method:
        case 'max':
            return signals.groupby(signals.date.dt.floor(floor)).max().drop('date', axis=1)
        case 'mean':
            return signals.groupby(signals.date.dt.floor(floor)).mean().drop('date', axis=1)
        case 'mixed': 
            pass
            # convolve, how to optimize params? perceptron?
        case _:
            raise ValueError(f'Unknown method: {method}')

In [ ]:
data_path = '../data/raw/'

files = os.listdir(data_path)

# make a dict, key - file_name_last part
dta = dict()
datasets = []
for file in files:
    df = pd.read_csv(os.path.join(data_path, file), skiprows=config.COUNT_SKIP, sep=';')
    df = loader.fill_empty(loader.transform_header(df))
    # Compress signal
    compressed = compress(df, floor='5min', method='mean')
    datasets.append(compressed)

min_length = min(df.shape[0] for df in datasets)
min_index = set(datasets[0].index)
for df in datasets[1:]:
    min_index.intersection_update(df.index)
min_index = list(min_index)

for i, df in enumerate(datasets):
    datasets[i] = df.loc[min_index]

# Проверка, что даты совпадают
for i in range(len(files)):
    for j in range(i + 1, len(files)):
        assert np.setdiff1d(datasets[i].index, datasets[j].index).size == 0, f"Intersection has shape {np.setdiff1d(datasets[i].index, datasets[j].index).shape}"

for i in range(len(datasets)):
    datasets[i].sort_index(inplace=True)

for i, file in enumerate(files):

    splitted = loader.split(datasets[i].columns)
    group = loader.group(splitted, datasets[i])

    dta[file] = group

In [ ]:
def exponential_moving_average(data, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    ema = np.convolve(data, weights, mode='full')[:len(data)]
    ema[:window] = ema[window]
    return ema

In [ ]:
def idx2time(rng: np.array):
    pass

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(14, 14))
for i, file in enumerate(files):
    j = 0 
    for k in dta[file].keys():
        for kk in dta[file][k].keys():
            print(i, j, file, k, kk)
            # print(dta[file][k][kk][len(dta[file][k][kk]) // 2:][0])
            if len(dta[file][k][kk]) == 2:
                sns.lineplot(exponential_moving_average(dta[file][k][kk][len(dta[file][k][kk]) // 2:][0], window=50), ax=ax[i][j])
            else:
                sns.lineplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j])
            # sns.kdeplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j], color='green')
            # sns.histplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j])
            # ax[i][j].set_xscale('log')
            # ax[i][j].set_yscale('log')
            ax[i][j].set_title(file + ' ' + k + ' ' + kk, fontsize=8)
            j += 1

функция для сжатия точек до заданного промежутка времени - часы/дни. DONE (перенести в лоадер)

In [ ]:
# В сгруппированном нет даты, а значит я должен сжимать по дате до группировки

объединение данных по компонентам (перенести в лоадер)

In [ ]:
dd = defaultdict(list)
for d in (dta.keys()):
    for k_outer, v_outer in dta[d].items():
        for k_inner, v_inner in v_outer.items():
            # print(k_inner, v_inner)
            dd[k_inner].append(v_inner)

for key in dd.keys():
    component_mat = np.array([])
    for row in dd[key]:
        data_row = np.array(row[len(row) // 2:])
        if component_mat.size == 0:
            component_mat = data_row
        else:
            component_mat = np.vstack([component_mat, data_row])

определяем функцию деградации

In [ ]:
def fit_exp_custom(x, f, t, b, e, s):
    return f + t * e ** (b * x + e - (s ** 2) / 2)

In [ ]:
tmp = np.array(np.arange(1, len(dd['ППН'][0][1]) + 1))
for i in range(4):
    if tmp.size == 0:
        tmp = dd['ППН'][i][1]
    else:
        tmp = np.vstack([tmp, dd['ППН'][i][1]])

Перед PCA нормализуем данные

In [ ]:
def tkeo_operator(data, k = 1):
    """
    Teager-Kaiser Energy operator
    """
    npnts = len(data[0])
    nsignals = len(data)
    filt_data = deepcopy(data)
    for i in range(nsignals):
        for n in range(k, npnts-k):
            filt_data[i][n] = data[i][n]**2-data[i][n-1]*data[i][n+1]
    return filt_data

def normilize(signal: np.ndarray):
    """
    MinMaxScaler + Teager-Kaiser Operator + MinMaxScaler
    """
    # scalers = [MinMaxScaler, StandardScaler]
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal = scaler.fit_transform(signal)
    print(f'norm1 max: {signal.max()}, min: {signal.min()}')
    signal = tkeo_operator(signal)
    print(f'tkeo max: {signal.max()}, min: {signal.min()}')
    signal = scaler.fit_transform(signal)
    print(f'norm2 max: {signal.max()}, min: {signal.min()}')
    return signal


In [ ]:
tmp = normilize(tmp)

In [ ]:
pca = PCA(n_components=2)
compressed = pca.fit_transform(tmp.T)

In [ ]:
sns.lineplot(exponential_moving_average(abs(compressed[:, 1]), window=100))

Curve fit (NON LINEAR LSTSQ)

In [ ]:
# xdata = np.arange(1, 4001)
# popt, pcov = curve_fit(fit_exp_custom, xdata, exponential_moving_average(abs(compressed[:, 1][:4000]), window=100))

# Чтобы так просто фитить нужно, чтобы в данных была экспонента, а у меня не так. Очевидно, что получится мусор

TODO:  
Проверка кусочно-заданной, байесовская оптимизация параметров  
Реализация индикатора здоровья через количество аномалий

Bayesian Ridge

Самописный байес для обновления параметров фукнции

Подсчёт числа аномалий (граница - параметр функции). Экспоненциальный закон снижения остаточного ресурса.

In [ ]:
...